In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neighbors import NearestNeighbors
import sys
import time
from statistics import mean
import math
from sklearn import preprocessing

In [ ]:
start = time.time()

In [ ]:
name_of_file1 = 'a3.csv'
thresh = 0.3                                                          #### Imp
K = 14 ## Actually K-1 neighbors are considered. Remember              #### Imp
alpha_weight_avg = 0.6


# K1 = 11 if 11 < K else K
thresh1 = thresh #1.0 - thresh 
# print(K1)
limit = sys.getrecursionlimit()

In [ ]:
data2 = np.genfromtxt(name_of_file1,delimiter=',')
#data1=data1/np.linalg.norm(data1)
print(len(data2)) #,"\n\n")

In [ ]:
# Normalization
# data1 = preprocessing.normalize(data2, axis=0, norm='l1') 
data1 = data2

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(data1[:,0],data1[:,1],c='blue', label='Datapoints')
plt.xlabel('X-Coordinates')
plt.ylabel('Y-Coordinates')
plt.title('Datapoints Visualization')
plt.legend()
plt.show()

In [ ]:
# Calculated the nearest neighbors, the density of point_i and density of its K neighbors
nbrs = NearestNeighbors(n_neighbors=K, algorithm='auto').fit(data1)
distances, indices = nbrs.kneighbors(data1)

In [ ]:
# print(indices[0][1:])

In [ ]:
# print(distances)

In [ ]:
density = []
for i in range(len(data1)):
    density.append(sum(distances[i])/(K-1))       

In [ ]:
# Getting max density
max_density = min(density)
idx_1 = density.index(max_density)

# Plotting       
plt.figure(figsize=(8,8))
plt.scatter(data1[:,0],data1[:,1],c='blue', label='Datapoints')
plt.scatter(data1[idx_1][0], data1[idx_1][1], c='red', s=200, label='Highest Density', marker = 'x')
plt.xlabel('X-Coordinates')
plt.ylabel('Y-Coordinates')
plt.title('Datapoints Visualization')
plt.legend()
plt.show()

In [ ]:
# print(min(density))
# print(density[484])
# print(indices[max_idx])

In [ ]:
# Weighted Average
def numpy_ewma_vectorized_v2(data, alpha):

    window = (2 /alpha) - 1.0
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    offset = data[0]*pows[1:]
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out[-1]

In [ ]:
def NovClus(idx, hash_map, anomaly_list, thresh, list_of_parents, cluster_no, c, res, new_iter, can_form_cluster):
    #print(can_form_cluster)
    #print(idx, hash_map[idx], len(anomaly_list), c)
    K = 0
    for key in hash_map: 
        if hash_map[key] == K: 
            res = res + 1
    print(res, cluster_no, c) ## Uncomment latter
        
    if c >= limit-50:
        return 
    
    if hash_map[idx] != 0 and new_iter == False:
        #print("returning 1")
        return
    
    if new_iter == True and data1[idx].tolist() in anomaly_list:
        anomaly_list.remove(data1[idx].tolist())
    
    if new_iter == False and can_form_cluster == False:
        can_form_cluster = True
        
    new_iter = False
    list_of_parents.append(density[idx])
    
    
    for index in indices[idx][1:]: 
        weighted_density = numpy_ewma_vectorized_v2(np.array(list_of_parents), alpha_weight_avg)
        numerator = abs(weighted_density - density[index])
        denominator = weighted_density
        if (numerator/denominator) > thresh: # 0.3 > 0.7      # >   #### Is this weighted thing giving correct results?
            hash_map[index] = -1
            list_val = data1[index].tolist()
            if list_val not in anomaly_list:
                anomaly_list.append(list_val)
        else:   
            #print("cluster number of element changed")
            hash_map[idx] = cluster_no
            #count += 1
            NovClus(index, hash_map, anomaly_list, thresh, list_of_parents, cluster_no, c+1, 0, new_iter, can_form_cluster)
    
    if can_form_cluster == False:
        hash_map[idx] = -1
        list_val = data1[idx].tolist()
        if list_val not in anomaly_list:
            anomaly_list.append(list_val)

In [ ]:
visited_hash_map = {i:False for i in range(len(data1))}
cluster_no = 0
anomaly_list = []
hash_map = {i:0 for i in range(len(data1))}


while (0 in hash_map.values()) == True:
           
    #print("Number of unvisited = ", count_0)
    min_val = float("inf")
    #max_idx = 0
    
    for i in range(len(data1)):
        if density[i] < min_val and hash_map[i] <= 0 and visited_hash_map[i] != True: #<=
            min_val = density[i]
            max_idx = i
    
    visited_hash_map[max_idx] = True
    
    list_of_parents = []
    
    new_iter = True
    res = 0
    can_form_cluster = False
    if hash_map[max_idx] != -1:
        cluster_no += 1
    #print("\n\nCalling function!!")
    NovClus(max_idx, hash_map, anomaly_list, thresh, list_of_parents, cluster_no, 0, res, new_iter, can_form_cluster)
    
    
    # This thing needs to be seen and thought of
    """count_0 = 0
    for key in hash_map:
        if hash_map[key] == 0:
            count_0 += 1
    #print("Number of unvisited = ", count_0)"""
    
    """plt.figure(figsize=(8,8))
    plt.scatter(data1[:,0],data1[:,1],c='blue', label='Datapoints')
    plt.scatter(data1[max_idx][0], data1[max_idx][1], c='red', s=200, label='Highest Density', marker = 'x')
    color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'lightgreen', 'orange', 'gray', 'purple', 'brown', 'olive', 'gold', 'deeppink', 'chocolate', 'lawngreen', 'pink', 'navy', 'blueviolet', 'paleturquoise', 'olive', 'lavender', 'navajowhite', 'khaki']
    for i in range(len(data1)):
        if hash_map[i] == cluster_no:
            plt.scatter(data1[i][0], data1[i][1], c=color[cluster_no])
    plt.xlabel('X-Coordinates')
    plt.ylabel('Y-Coordinates')
    plt.title('Datapoints Visualization')
    plt.legend()
    plt.show()"""
    
    """if count_0 <= 1:
        for key in hash_map:
            if hash_map[key] == 0:
                hash_map[key] == -1
                list_val = data1[key].tolist()
                if list_val not in anomaly_list:
                    anomaly_list.append(list_val)
        break"""
        
            
print("Number of Anomalies = ",len(anomaly_list))

In [ ]:
#store_anomaly_list = anomaly_list
#anomaly_list = store_anomaly_list
#print(anomaly_list)

In [ ]:
#print(sorted(anomaly_list))
def restore(anomaly_list, needs_iteration):
    #print("\n\n")
    for ele in anomaly_list:
        restore_list = []
        reqd_idx = data1.tolist().index(ele)
        #print(reqd_idx)
        #print(data1[reqd_idx])
        #print(hash_map[reqd_idx])
        #print(indices[reqd_idx][1:])
        #print(ele)
        #print(hash_map[indices[reqd_idx][1]]) ##
        #print(indices[reqd_idx][1])
        #print((abs(density[indices[reqd_idx][1]] - density[reqd_idx]) / density[reqd_idx])) ##
        for p in range(1,K):
            if hash_map[indices[reqd_idx][p]] > 0 and (abs(density[indices[reqd_idx][p]] - density[reqd_idx]) / density[reqd_idx]) <= thresh1: # thresh1
                # remove from anomaly list
                anomaly_list.remove(ele)
                # Assign cluster
                hash_map[reqd_idx] = hash_map[indices[reqd_idx][p]]
                break
                
    """for ele in anomaly_list:
        reqd_idx = np.where(data1 == ele)[0][0]
        for p in range(1,K):
            if hash_map[indices[reqd_idx][p]] < 0 and hash_map[reqd_idx] == -1 and (abs(density[indices[reqd_idx][p]] - density[reqd_idx]) / density[reqd_idx]) <= thresh1:
                needs_iteration = True
                break
    print(needs_iteration)
    #return needs_iteration """
    return len(anomaly_list)
    

runs = 0
stored_val = float("inf")
while runs < len(data1)/2:
    if anomaly_list == 0:
        break
    if restore(anomaly_list, False) == stored_val:
        break
    else:
        stored_val = restore(anomaly_list, False)
    runs += 1
    

In [ ]:
print(len(anomaly_list))
#print(len(store_anomaly_list))

In [ ]:
"""# print(hash_map)
cluster_idx = cluster_no
while cluster_idx > 0:
    count_f = 0
    for key in hash_map:
        if hash_map[key] == cluster_idx:
            count_f += 1
    if count_f <=1:
        cluster_no -= 1
    cluster_idx -= 1
        
print(cluster_no)"""

In [ ]:
#labels = [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 29, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 29, 5, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 7, 7, 28, 7, 25, 25, 7, 7, 28, 7, 7, 25, 7, 7, 7, 7, 9, 7, 25, 25, 7, 7, 25, 25, 7, 28, 9, 25, 7, 7, 7, 25, 7, 7, 7, 7, 28, 25, 7, 7, 7, 7, 7, 28, 7, 25, 25, 7, 7, 25, 7, 7, 14, 9, 25, 25, 7, 7, 25, 25, 25, 25, 25, 7, 7, 25, 7, 7, 7, 25, 7, 25, 25, 25, 28, 7, 7, 25, 7, 7, 28, 7, 7, 7, 7, 28, 7, 25, 7, 7, 7, 25, 25, 7, 9, 25, 7, 7, 25, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 21, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 26, 0, 26, 0, 27, 26, 26, 28, 27, 0, 27, 0, 0, 0, 26, 0, 0, 0, 27, 26, 26, 22, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 28, 0, 0, 0, 26, 0, 26, 0, 27, 0, 26, 22, 0, 0, 0, 26, 0, 0, 0, 0, 26, 0, 26, 0, 0, 26, 0, 0, 26, 0, 0, 0, 26, 0, 27, 27, 26, 0, 26, 0, 26, 0, 26, 26, 0, 27, 0, 26, 0, 27, 0, 26, 0, 0, 0, 0, 0, 0, 0, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 24, 2, 2, 24, 24, 2, 24, 24, 24, 24, 18, 24, 2, 2, 2, 24, 2, 2, 2, 2, 2, 24, 2, 24, 24, 2, 2, 24, 24, 24, 2, 2, 2, 24, 24, 2, 24, 24, 2, 24, 2, 2, 2, 2, 24, 2, 2, 2, 24, 24, 2, 2, 2, 2, 2, 2, 2, 2, 2, 24, 24, 24, 2, 2, 2, 24, 2, 18, 24, 24, 2, 24, 24, 24, 2, 2, 10, 2, 2, 2, 24, 2, 2, 24, 30, 2, 2, 2, 24, 2, 2, 24, 2, 24, 24, 2, 2, 2, 2, 19, 19, 28, 19, 19, 26, 19, 19, 19, 26, 19, 26, 19, 19, 19, 26, 19, 26, 19, 19, 19, 19, 19, 19, 19, 19, 19, 28, 19, 19, 26, 19, 19, 19, 28, 19, 26, 19, 19, 19, 19, 19, 19, 26, 19, 19, 19, 19, 19, 19, 19, 28, 19, 19, 19, 19, 19, 19, 26, 28, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 26, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 26, 19, 19, 19, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 27, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 27, 13, 13, 13, 13, 13, 27, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 27, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 27, 13, 13, 13, 13, 13, 13, 13, 27, 4, 4, 26, 4, 4, 4, 4, 4, 4, 4, 4, 4, 26, 4, 4, 4, 26, 4, 4, 4, 4, 4, 4, 4, 4, 29, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 29, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 26, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 26, 4, 13, 13, 13, 13, 35, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 23, 13, 13, 13, 13, 20, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 20, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 20, 16, 13, 13, 13, 13, 13, 13, 13, 13, 13, 35, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 26, 5, 5, 5, 26, 5, 5, 5, 5, 5, 5, 5, 5, 26, 5, 5, 5, 5, 5, 5, 5, 5, 5, 18, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 24, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 26, 5, 5, 5, 18, 5, 5, 5, 5, 26, 5, 26, 26, 5, 5, 5, 5, 5, 5, 5, 24, 5, 5, 18, 5, 18, 5, 26, 26, 5, 18, 5, 5, 5, 5, 5, 5, 5, 27, 10, 10, 10, 10, 10, 10, 10, 10, 10, 25, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 30, 10, 10, 10, 10, 10, 10, 10, 10, 10, 27, 10, 10, 10, 10, 10, 10, 30, 10, 10, 10, 10, 10, 11, 10, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 25, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 25, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 14, 14, 25, 14, 14, 14, 25, 14, 14, 25, 25, 14, 14, 25, 14, 14, 14, 14, 25, 14, 14, 27, 14, 14, 25, 14, 14, 14, 14, 25, 27, 14, 14, 14, 14, 14, 14, 25, 25, 14, 14, 14, 14, 14, 14, 25, 25, 27, 14, 14, 14, 25, 14, 14, 25, 14, 14, 14, 14, 25, 25, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 27, 14, 14, 14, 14, 14, 14, 14, 25, 14, 14, 14, 25, 14, 14, 14, 14, 14, 14, 25, 14, 14, 14, 14, 9, 7, 9, 9, 9, 9, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 27, 9, 9, 9, 9, 27, 9, 9, 27, 9, 27, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 9, 27, 9, 9, 27, 9, 27, 9, 9, 9, 27, 9, 9, 27, 9, 27, 27, 9, 9, 9, 9, 9, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 28, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 28, 28, 24, 24, 24, 24, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 28, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 17, 17, 17, 17, 17, 25, 17, 17, 17, 17, 17, 17, 17, 17, 17, 28, 17, 17, 17, 17, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 25, 10, 10, 10, 10, 10, 10, 10, 27, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 27, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 25, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 27, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 27, 12, 12, 27, 12, 28, 12, 12, 12, 12, 27, 12, 28, 12, 27, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 4, 12, 12, 12, 12, 12, 27, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 28, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 22, 22, 22, 22, 22, 22, 22, 22, 27, 22, 22, 27, 22, 22, 22, 22, 28, 22, 22, 22, 22, 28, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 27, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 26, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 5, 5, 5, 18, 5, 5, 18, 5, 5, 5, 5, 5, 5, 5, 5, 18, 5, 5, 5, 5, 18, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 18, 18, 5, 5, 5, 18, 5, 5, 5, 5, 18, 18, 5, 5, 5, 5, 5, 18, 18, 5, 18, 18, 18, 18, 5, 18, 18, 18, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 18, 18, 5, 18, 18, 5, 5, 5, 5, 5, 5, 18, 18, 18, 5, 5, 5, 18, 18, 5, 5, 5, 5, 5, 5, 5, 5, 25, 25, 25, 10, 6, 6, 6, 6, 25, 6, 25, 25, 6, 25, 25, 6, 25, 6, 25, 25, 25, 25, 25, 6, 25, 6, 6, 25, 28, 25, 25, 25, 25, 25, 6, 25, 25, 6, 25, 28, 25, 28, 25, 25, 25, 25, 6, 25, 6, 28, 25, 25, 6, 25, 25, 25, 25, 25, 25, 28, 25, 25, 25, 25, 6, 6, 25, 6, 25, 25, 25, 25, 28, 25, 25, 28, 25, 28, 25, 6, 25, 25, 6, 6, 25, 25, 25, 25, 25, 25, 25, 25, 33, 25, 25, 25, 25, 25, 25, 25, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 26, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 26, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 16, 16, 16, 16, 27, 20, 16, 16, 28, 16, 16, 16, 16, 16, 16, 16, 20, 27, 16, 16, 16, 16, 16, 20, 16, 16, 16, 16, 16, 16, 16, 28, 16, 16, 16, 16, 16, 16, 20, 16, 16, 16, 16, 20, 16, 16, 16, 16, 16, 16, 16, 28, 16, 16, 16, 16, 16, 20, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 20, 16, 16, 16, 16, 20, 20, 16, 27, 16, 16, 16, 16, 16, 23, 23, 23, 23, 23, 28, 23, 27, 23, 28, 23, 28, 23, 27, 23, 23, 28, 23, 23, 23, 23, 23, 23, 23, 23, 23, 28, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 28, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 28, 28, 23, 27, 23, 23, 23, 23, 23, 23, 23, 27, 23, 23, 23, 23, 23, 23, 23, 23, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 27, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 27, 21, 21, 21, 21, 21, 27, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 27, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 31, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 31]

def get_count_in_label(labels):
    table = {value:0 for value in labels}
    for ele in labels:
        table[ele] += 1
    key_list = table.keys()
    #sorted(key_list, reverse = False)
    for ele in sorted(key_list, reverse = False):
        print(ele,":",table[ele])

def get_label_list(hash_map):
    label_list = []
    for key in hash_map.keys():
        if hash_map[key] > 0:
            label_list.append(hash_map[key])
        #else:
        #    get_labels.append(hash_map[key])
    #print(get_labels)
    return label_list    

        
def find_missing_idx(hash_map):
    max_idx = float("-inf")
    for key in hash_map:
        if hash_map[key] > max_idx:
            max_idx = hash_map[key]
    #max_idx = max(hash_map.keys())
    
    min_idx = float("inf")
    for key in hash_map:
        if hash_map[key] < min_idx and hash_map[key] != -1:
            min_idx = hash_map[key]
    #min_idx = 0

    #print(min_idx, max_idx)                     ###############
    
    label_list = get_label_list(hash_map)

    missing_idx = []
    count = 1
    while count <= max_idx:
        if count not in label_list:
            missing_idx.append(count)
        count += 1

    #print(missing_idx)                           ##########
    return missing_idx

def correct_the_idx(hash_map): #labels):
    missing_idx = find_missing_idx(hash_map) #labels)
    while missing_idx:
        for i in missing_idx:
            for key in hash_map:# in range(len(labels)):
                if hash_map[key] == i + 1: #labels[j] == i+1:
                    hash_map[key] = hash_map[key] - 1 #labels[j] = labels[j] - 1

        #get_count_in_label(labels)
        
        #print("\n")
        #coorect_the_idx(labels)
        
        missing_idx = find_missing_idx(hash_map) #labels)

    
#missing_idx = correct_the_label_idx(labels)
#print(missing_idx)

#get_count_in_label(labels)
# print("\n\n")
correct_the_idx(hash_map) #labels)
#get_count_in_label(labels)
#find_missing_idx(hash_map)
#print(hash_map)

#label_l = get_label_list(hash_map)
#get_count_in_label(label_l)


In [ ]:
get_labels = []
for key in hash_map.keys():
    if hash_map[key] > 0:
        get_labels.append(hash_map[key]-1)
    #else:
    #    get_labels.append(hash_map[key])
print(get_labels)

In [ ]:
cluster_count = set()
for key in hash_map.keys():
    if hash_map[key] > 0:
        cluster_count.add(hash_map[key])
print(len(cluster_count))

In [ ]:
cluster_idx = cluster_no
while cluster_idx > 0:
    count_f = 0
    for key in hash_map:
        if hash_map[key] == cluster_idx:
            count_f += 1
    if count_f <=1:
        cluster_no -= 1
    cluster_idx -= 1
        
print(cluster_no)

In [ ]:
plt.figure(figsize=(8,8))
# plt.scatter(data1[:,0],data1[:,1],c='blue', label='Datapoints')

color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'lavenderblush', 'navajowhite', 'khaki']

if cluster_no <= 23:
    cluster = 0
    while cluster < cluster_no:
        for i in range(len(data1)):
            if hash_map[i] == cluster+1:
                plt.scatter(data1[i][0], data1[i][1], c=color[cluster])
        cluster += 1
else:
    plt.scatter(data1[:,0],data1[:,1],c='blue', label='Datapoints')
    
for ele in anomaly_list:
    plt.scatter(ele[0],ele[1],c='green',marker='*',s=150)  
    
plt.xlabel('X-Coordinates')
plt.ylabel('Y-Coordinates')
plt.title('Datapoints Visualization')
#plt.legend()
plt.show()

In [ ]:
print("time taken = ",time.time()-start," sec")
#print("Average density =", sum(density)/len(data1))